# Calculate Accretion Rates using the scaling relations from Aoyama+2021 for Delorme 1 ABb

written by Sarah Betti 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy import units as u
from astropy import constants as const

from scipy.interpolate import interp1d

# line fluxes from Table 1 Betti2022b erratum

[line flux in erg/s/cm2, uncertainty in erg/s/cm2]

In [3]:
# epoch 1: UT 2021-11-20
Del1PaB = [8.05e-16, 0.56e-16] 
Del1PaG = [6.82e-16, 0.61e-16] 
Del1BrG = [1.64e-16, 0.29e-16] 

# epoch 2: UT 2022-01-24
Del2PaB = [3.49e-16, 0.52e-16] 
Del2PaG = [2.94e-16, 0.62e-16] 
Del2BrG = [0.64e-16, np.nan] 


# functions to measure Mdot using Aoyama+2021 scaling relations

In [4]:
def measure_line_luminosity(Line_Flux, Distance):
    line_flux, line_flux_err = Line_Flux 
    distance, distance_err = Distance
    # line luminosity
    if isinstance(line_flux, float):
        line_flux = line_flux * u.erg / u.s/ u.cm**2
    if isinstance(line_flux_err, float):
        line_flux_err = line_flux_err * u.erg / u.s/ u.cm**2

    dist = distance * u.pc
    distance_err = distance_err * u.pc
    Line_Luminosity = 4 * np.pi * dist**2. * line_flux
    Line_Luminosity_err = Line_Luminosity * np.sqrt((2*distance_err/dist)**2. + (line_flux_err/line_flux)**2.)

    # convert to Lsun
    Line_Luminosity_Lsun = Line_Luminosity.to(u.Lsun)
    Line_Luminosity_Lsun_err = Line_Luminosity_err.to(u.Lsun)
    if not np.isnan(Line_Luminosity_Lsun_err.value):
        print('Lline_Lsun: ', Line_Luminosity_Lsun, '+/-', Line_Luminosity_Lsun_err)
    else: 
        print('Lline_Lsun: <', Line_Luminosity_Lsun)
    return [Line_Luminosity_Lsun, Line_Luminosity_Lsun_err]

def Aoyama_scaling(line):
    # taken from Aoyama+2021 Table 1
    if line == 'PaB':
        a, aerr, b, berr = 0.86, 0, 2.21, 0
    elif line == 'PaG':
        a, aerr, b, berr = 0.85, 0, 2.28, 0
    else:
        a, aerr, b, berr = 0.85, 0, 2.84, 0
    return a, aerr, b, berr

def measure_accretion_luminosity(line, Line_Luminosity):
    # line = either PaB, PaG, or BrG
    
    line_lum, line_lum_err = Line_Luminosity
    line_lum_err_LOG = 0.434 * line_lum_err/line_lum

    # scaling relation
    a,a_err,b,b_err = Aoyama_scaling(line)
    Lacc_Lsun = 10**(a * np.log10(line_lum.value) + b)*u.Lsun

    #uncertainty propagation
    log10L = np.log10(line_lum.value)
    sigma_log10L = 0.434 * line_lum_err.value/line_lum.value

    alog10L = a * log10L
    alog10Lb = a * log10L + b 

    logsigma_Lacc = 0.3 # from Aoyama+2021
    sigma_Lacc = 2.303 * (10**(alog10Lb)) * logsigma_Lacc
    Lacc_Lsun_err = sigma_Lacc * u.Lsun

    if not np.isnan(Lacc_Lsun_err.value):
        print('LOG Lacc_Lsun: ', np.log10(Lacc_Lsun.value)*Lacc_Lsun.unit, '+/-', logsigma_Lacc*Lacc_Lsun.unit)
        print('Lacc_Lsun: ', Lacc_Lsun, '+/-', Lacc_Lsun_err)
    else:
        print('LOG Lacc_Lsun: <', np.log10(Lacc_Lsun.value)*Lacc_Lsun.unit)
        print('Lacc_Lsun: <', Lacc_Lsun)
    return [Lacc_Lsun, Lacc_Lsun_err], [np.log10(Lacc_Lsun.value)*Lacc_Lsun.unit, logsigma_Lacc*Lacc_Lsun.unit]


def measure_Mdot(mass, radius, acc_lum, Rin=5*u.Rsun):
    Lacc, Lacc_err = acc_lum
    # convert to Mdot
    Radius_Rsun = radius[0] * u.Rsun
    Radius_Rsun_err = radius[1] * u.Rsun
    Mass_Msun = mass[0] * u.Msun
    Mass_Msun_err = mass[1] * u.Msun
    G = const.G.cgs

    Mdot = ((1-(Radius_Rsun/Rin))**-1) * (Radius_Rsun*Lacc)/(const.G*Mass_Msun)

    Mdot_err = Mdot * np.sqrt((Lacc_err/Lacc)**2. + (Radius_Rsun_err/Radius_Rsun)**2. + (Mass_Msun_err/Mass_Msun)**2.)

    LOGmass = np.log10(Mass_Msun.value)*u.Msun
    LOGmass_err = 0.434 * Mass_Msun_err.value/Mass_Msun.value  *u.Msun

    LOGMdot = np.log10(Mdot.to(u.Mjup/u.yr).value)*u.Mjup/u.yr
    LOGMdot_err = 0.434 * Mdot_err.to(u.Mjup/u.yr) / Mdot.to(u.Mjup/u.yr)
    if not np.isnan(Mdot_err.value):
        print('Mdot:', Mdot.to(u.Mjup/u.yr), '+/-',Mdot_err.to(u.Mjup/u.yr))
        print()
        print('Log(Mdot): ', LOGMdot , '+/-',LOGMdot_err )
    else:
        print('Mdot: <', Mdot.to(u.Mjup/u.yr))
        print()
        print('Log(Mdot): <', LOGMdot)
    print('Log(Mass):', LOGmass, '+/-', LOGmass_err)

    return [LOGmass, LOGmass_err],[ LOGMdot, LOGMdot_err]




# measure Mdot for line fluxes in Mjup/year

In [13]:
# measure for each individual line 
# does each line for all epochs
# has manually inputted distances, masses, radii for Delorme 1 ABb
line = ['PaG', 'PaB', 'BrG', 'PaG', 'PaB',  'BrG']
LACC1 = []
LACC2 = []
for i, lineflux in enumerate([Del1PaG, Del1PaB, Del1BrG, Del2PaG, Del2PaB, Del2BrG]):
    if i == 0:
        print('   ==> EPOCH 1')
    if i == 3:
        print('============================')
        print('   ==> EPOCH 2')
    print(' ', line[i])
    Lline = measure_line_luminosity(lineflux, [47.2, 3])
    Lacc = measure_accretion_luminosity(line[i], Lline)
    if i <= 2:
        LACC1.append(Lacc[0][0].value)
    else:
        LACC2.append(Lacc[0][0].value)
    Mdot = measure_Mdot([0.012, 0.00095], [0.163, 0.01], Lacc[0], Rin=5*u.Rsun)
    print()

    
# find average Mdot for each epoch, exclude upper limit  
print('=========AVG===============')
Mdot1 = measure_Mdot([0.012, 0.00095], [0.163, 0.01], [np.mean(LACC1)*u.Lsun, np.std(LACC1)*u.Lsun], Rin=5*u.Rsun)
Mdot2 = measure_Mdot([0.012, 0.00095], [0.163, 0.01], [np.mean(LACC2[0:2])*u.Lsun, np.std(LACC2[0:2])*u.Lsun], Rin=5*u.Rsun)
# print('============================')
# print(Mdot1)
# print(Mdot2)

   ==> EPOCH 1
  PaG
Lline_Lsun:  4.7490571651672226e-08 solLum +/- 7.3815637366154735e-09 solLum
LOG Lacc_Lsun:  -3.9448837123067118 solLum +/- 0.3 solLum
Lacc_Lsun:  0.00011353147694810031 solLum +/- 7.84388974234425e-05 solLum
Mdot: 5.318364653387939e-08 jupiterMass / yr +/- 3.712865955733938e-08 jupiterMass / yr

Log(Mdot):  -7.274221888605645 jupiterMass / yr +/- 0.30298483270830917
Log(Mass): -1.9208187539523751 solMass +/- 0.03435833333333333 solMass

  PaB
Lline_Lsun:  5.6055586773601374e-08 solLum +/- 8.12293017745723e-09 solLum
LOG Lacc_Lsun:  -4.026187743422054 solLum +/- 0.3 solLum
Lacc_Lsun:  9.414825101637722e-05 solLum +/- 6.504702662721502e-05 solLum
Mdot: 4.4103604026281806e-08 jupiterMass / yr +/- 3.078968453395508e-08 jupiterMass / yr

Log(Mdot):  -7.3555259197209875 jupiterMass / yr +/- 0.3029848327083092
Log(Mass): -1.9208187539523751 solMass +/- 0.03435833333333333 solMass

  BrG
Lline_Lsun:  1.1420020162572207e-08 solLum +/- 2.4870418020733897e-09 solLum
LOG Lacc